## run notebooks

In [41]:
%run "../Notebooks/intialization.ipynb"
%run "../Notebooks/Stitching_functions.ipynb"
%run "../Notebooks/Visualization_functions.ipynb"
%run "../Notebooks/Downsampling_and_outliers_functions.ipynb"
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
## show files in directory recenlty modified
#!find . -mtime 0

## Load the pointclouds

In [42]:
## parameters initialization 
myparams = "./ScreenCamera_2021-01-12-10-28-18.json"  # parameter for camera point view, json file via pressing P
myconfiguration_file = "RenderOption_2021-01-12-10-24-27.json" # configuration file for properties, json file via pressing o


In [43]:
stitched_external_pc = o3d.io.read_point_cloud("stitched_external_pc.pcd")
stitched_internal_pc = o3d.io.read_point_cloud("stitched_internal_pc.pcd")
stitched_upper_pc = o3d.io.read_point_cloud("stitched_upper_pc.pcd")

In [44]:
labels_stitches = ["stitched_external_pc","stitched_internal_pc", "stitched_upper_pc"]
list_stitches = [stitched_external_pc,stitched_internal_pc,stitched_upper_pc]
color_stitches = [[1, 0.706, 0],[0, 0.651, 0.929],[0,0.7,0.3]]

## Many plots

In [45]:
# plot a list of geometries
custom_draw_geometry([list_stitches[:]],
                     mytitle = [labels_stitches[:]],
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


### downsampling

In [46]:
downsampled_stitches = downsample_list_stitches(list_stitches,
                                                #labels_stitches,color_stitches , 
                                                voxel_size = 0.3,
                                                visualization_on = True,
                                                print_statements = True)

Downsampled_pcl_1
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_pcl_1
number of points original :  230502
number of points with voxel down sample : 16013

Downsampled_pcl_2
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_pcl_2
number of points original :  227230
number of points with voxel down sample : 11910

Downsampled_pcl_3
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_pcl_3
number of points original :  219648
number of points with voxel down sample : 13335



In [24]:
## alignment check
#i = 0
#o3d.visualization.draw_geometries([downsampled_stitches[i].paint_uniform_color([0.5, 0, 0]),
                                   #list_stitches[i].paint_uniform_color([0, 0.5, 0])])

## Merge the external, upper, internal together

### Step 1- 

In [47]:
# timestamp
now = datetime.now()
dt_string = now.strftime("%d%m%Y-%Hh%Mm%Ss")
print(dt_string)

# initialize 
voxel_size  =0.2 # with 0.1 doesnt work!
i = 2
j = 0
source_label = labels_stitches[i][9:-3] # just labels parsing
target_label =labels_stitches[j][9:-3]

source = list_stitches[i]
target = list_stitches[j]

processed_source = downsampled_stitches[i]
processed_target = downsampled_stitches[j]

mytitle= dt_string+"-%s_%s"%(source_label,target_label)
start = timer()

#MAIN
source_output, target_output, transformation_1 = stitch_sequences(processed_source,processed_target,
                                                  source_label,target_label,
                                                  mmax_iteration = 10**7,
                                                  mmax_validation = 0.999,
                                                  print_statements = True, 
                                                  visualization_on = False  
                                                 )

end = timer()
print (f"duration: {end-start}")
draw_registration_result(source_output, target_output, transformation_1, title = dt_string+"Downsampled Registration results")


## apply transformation on original 
draw_registration_result(source, target, transformation_1, title = dt_string+"Full Registration results")

12012021-13h51m53s
allpc_downsampled_upper_external
Load two point clouds and disturb initial pose.
removing outliers

GLOBAL REGISTRATION: RANSAC registration on downsampled point clouds.
Transformation is:
[[  0.99522317  -0.07050567   0.06752629  -3.54340418]
 [  0.04112075   0.93007491   0.3650613  -24.10631397]
 [ -0.0885434   -0.36054073   0.92853134   5.29086722]
 [  0.           0.           0.           1.        ]]
overlapping points :  15
RegistrationResult with fitness=4.301692e-03, inlier_rmse=2.048818e-01, and correspondence_set size of 15
Access transformation to get result.

POINT-TO-PLANE ICP registration is applied on original point
distance threshold 0.400.
Transformation is:
[[  0.98764626  -0.02989253   0.15382231  -7.23425995]
 [ -0.0487185    0.87441172   0.48273248 -27.85461593]
 [ -0.14893413  -0.48426292   0.86215315   9.09004466]
 [  0.           0.           0.           1.        ]]
overlapping points :  3144
RegistrationResult with fitness=2.357705e-01, in

In [49]:
upper_external = save_registration_result(source, target, transformation_1, 
                                        title= mytitle, 
                                        save_result = False,
                                        visualize_result = False)

## downsample new one for next step

In [51]:
downsampled_upper_external = downsample_list_stitches([upper_external],
                                                        ["upper_external"],color_stitches= None , 
                                                        voxel_size = 0.3,
                                                        visualization_on = True,
                                                        print_statements = False)

Downsampled_upper_external
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


## Step 2-

In [56]:
# input 
voxel_size  =0.2
j = 1
source_label = "upper_external"
target_label =labels_stitches[j][9:-3]

processed_source = downsampled_upper_external[0]
processed_target = downsampled_stitches[j]

mytitle= dt_string+"-%s_%s"%(source_label,target_label)


start = timer()

source_output, target_output,transformation_2 = tqdm(stitch_sequences(processed_source,processed_target,
                                                                  source_label,target_label,
                                                                  mmax_iteration = 10**7,
                                                                  mmax_validation = 0.999,
                                                                  print_statements = False,
                                                                  visualization_on = False
                                                                 )
                                                )


end = timer()
print (f"duration: {end-start}")
draw_registration_result(source_output, target_output, transformation_2, title = "Downsampled Registration results")

source = upper_external
target = list_stitches[j]
draw_registration_result(source, target, transformation_2, title = "Registration results")

allpc_downsampled_upper_external_internal


  0%|          | 0/3 [00:00<?, ?it/s]

duration: 15.386144336998768


In [57]:
mytitle

'12012021-13h51m53s-upper_external_internal'

In [59]:
#save it
complete = save_registration_result(source, target, transformation_2, 
                                        title= mytitle, 
                                        save_result = False,
                                        visualize_result = False)

custom_draw_geometry(complete,
                     mytitle = "complete stitch",
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


In [60]:
get_num_points([complete])

[677380]

In [61]:
o3d.io.write_point_cloud("complete.pcd", complete)
o3d.io.write_point_cloud(dt_string+"complete.pcd", complete)

True